建物データと浸水面標高ラスターデータから建物の被災データを生成。（必要手続き：Driveの接続）

In [1]:
# @title Googleドライブの準備
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title 設定するパラメータ

# ケース名：解析を通して使います。
casename = "kurume-r2-rev2"

# 浸水範囲を求める際に設定した最大水深
maxdepth = 4.27

# Re:Earthへのアップロード用
# 片方でも None の場合はアップロードされません。
integrationKey = None # インテグレーションキー（文字列）
projectId = None # プロジェクトページのURLより（文字列）

# 確認用画像出力用フラグ
flg_checkDips = False

In [3]:
# buildings_json_path = "/content/drive/MyDrive/PLATEAU-FloodSAR/kurume-r2-dev/buildings_parsed.json"

In [4]:
# @title 詳細設定用パラメータ（基本的には操作不要）

enable_savefig = False # 図をGoogleドライブに保存する？

zoomlevel = 15 # DEM zoom level to fetch

# デバッグ用出力
global globalflag_debug

# 保存先
path_home = "/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/"
path_cgml = path_home + "/CityGML/"
path_dem = path_home + "/DEM/"
path_case = path_home + casename + "/"
path_upload = path_case + "ForUpload/"
file_flbbox = path_case + "floodprb_bbox.npy"
json_boundary = path_case +"boundary.json"
file_bbox = path_case + "boundbox.npy"

file_flooddem = path_case + "flood_dem_{:04d}.npz"
file_flooddepth = path_case + "flood_depth_{:04d}.npz"
file_building_csv = path_upload + "flood_buildings.csv"

path_flood3Dtile_tmp =  "flood3dtile_depth/"
path_deltphtile_tmp = "ellipsoid_xyztiles/"
file_flood3Dtile_zip = path_upload+"flood_ellipsoid_3dtile.zip"
file_floodXYZtile_zip = path_upload+"flood_depth_xyztile.zip"

In [5]:
#@title ライブラリインポート
#!pip install plateauutils
!pip install git+https://github.com/eukarya-inc/plateauutils.git@main#egg=plateauutils
!pip install reearthcmsapi
import matplotlib.pyplot as plt
import numpy as np
from shapely import box
from scipy import interpolate
from progressbar import progressbar
import zipfile
import os
import sys
import shutil
import plateauutils
from plateauutils.flood_converter.flood_to_3dtiles import FloodTo3dtiles
from reearthcmsapi.apis.tags import items_api
from reearthcmsapi.model.versioned_item import VersionedItem
from reearthcmsapi.model.asset_embedding import AssetEmbedding
from reearthcmsapi.apis.tags import assets_project_api
from reearthcmsapi.model.asset import Asset
from pprint import pprint
sys.path.append(path_home)
import plateau_floodsar_lib as pfsl

  Cloning https://github.com/eukarya-inc/plateauutils.git (to revision main) to /tmp/pip-install-ia5uvf2t/plateauutils_1566d812c5dc4b7dbca3b98e9f5936d2
  Running command git clone --filter=blob:none --quiet https://github.com/eukarya-inc/plateauutils.git /tmp/pip-install-ia5uvf2t/plateauutils_1566d812c5dc4b7dbca3b98e9f5936d2
  Resolved https://github.com/eukarya-inc/plateauutils.git to commit 988d053eae5ff3b0bd9e23445fdb6171796a418f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://gitlab.com/Oslandia/py3dtiles.git (to revision 489289d2f7310fbcb8e95710e250d893999adf5d) to /tmp/pip-install-ia5uvf2t/py3dtiles_704d7ed9e1284415b02a9079ce10d817
  Running command git clone --filter=blob:none --quiet https://gitlab.com/Oslandia/py3dtiles.git /tmp/pip-install-ia5uvf2t/py3dtiles_704d7ed9e1284415b02a9079ce10d817
  Running command git rev-parse -q --verify 'sha^489289d2f7310fbcb8e95710e250d89399

/usr/local/lib/python3.10/dist-packages/py3dtiles/tilers/node/distance.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(cache=True, nogil=True)


In [6]:
if not os.path.exists(path_upload):
  os.mkdir(path_upload)

## 浸水データ読み込みと楕円体高への変換、浸水カテゴリラスターの生成

In [7]:
# @title 領域の読み込み
boundbox = np.load(file_flbbox)
print(boundbox)

[130.40008279  33.22512157 130.6999996   33.36663976]


In [8]:
# @title ジオイド高マップ取得
bbox = box(*boundbox)
print(bbox)
ggh = pfsl.GiajGeoidHandler(path_home)
ggh.load_dem_tiles(bbox)
tile = ggh.produce_tile_stiched()
geo_data = tile["geoid"]
geo_lons = tile["lons"]
geo_lats = tile["lats"]
idx_l = np.where(boundbox[0] <= geo_lons)[0][0]-1
idx_h = np.where(boundbox[2] < geo_lons)[0][0]+1
#print(idx_l, idx_h)
idy_l = np.where(boundbox[3] >= geo_lats)[0][0]-1
idy_h = np.where(boundbox[1] > geo_lats)[0][0]+1
#print(idy_l, idy_h)
geo_data = geo_data[idy_l:idy_h+1,idx_l:idx_h+1]
geo_lons = geo_lons[idx_l:idx_h+1]
geo_lats = geo_lats[idy_l:idy_h+1]

POLYGON ((130.69999959514536 33.225121566010124, 130.69999959514536 33.366639756095395, 130.40008279106954 33.366639756095395, 130.40008279106954 33.225121566010124, 130.69999959514536 33.225121566010124))


100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [9]:
#@title ジオイド高の確認
if flg_checkDips:
  print(boundbox)
  boxline_x = [boundbox[0],boundbox[2],boundbox[2],boundbox[0],boundbox[0]]
  boxline_y = [boundbox[1],boundbox[1],boundbox[3],boundbox[3],boundbox[1]]
  ax =plt.subplot(1,1,1)
  #img = ax.contourf(geo_data)#,levels=np.arange(1,cnt+10,10))
  img = ax.contourf(geo_lons, geo_lats, geo_data)#,levels=np.arange(1,cnt+10,10))
  plt.plot(boxline_x,boxline_y, "r-")
  ax.axis("equal")
  plt.colorbar(img)

In [10]:
#@title 浸水DEMラスターの読み込みと確認
dem_map = np.load(file_flooddem.format(int(maxdepth*100)))
lons = dem_map["lons"]
lats = dem_map["lats"]
demmap = dem_map["floodmap_dem"]

if flg_checkDips:
  ax =plt.subplot(1,1,1)
  img = ax.contourf(lons,lats, demmap)#,levels=np.arange(0,maxdepth+0.1,0.25))
  ax.axis("equal")
  plt.colorbar(img)

In [11]:
#@title 楕円体高への変換
print(len(geo_lats), len(geo_lons), np.shape(geo_data))
geoInterp = interpolate.RectBivariateSpline(geo_lats[::-1], geo_lons, geo_data[::-1,::])
tmp = geoInterp(lats[::-1],lons)[::-1,:]
print(np.shape(tmp), np.max(tmp), np.min(tmp))
geomap = demmap + tmp
#print("\nUp to ", len(lons))
#for ii, lon in progressbar(enumerate(lons)):
#  for jj,lat in enumerate(lats):
#    demmap[jj,ii] += ggh.calc_dem_interp(lon, lat)

34 58 (34, 58)
(4096, 7168) 33.05213467648333 32.44529952016204


In [12]:
#@title 浸水ジオイド高確認用
if flg_checkDips:
  print((len(lons), len(lats), np.shape(geomap)))
  ax =plt.subplot(1,1,1)
  img = ax.contourf(lons,lats, geomap)#,levels=np.arange(0,maxdepth+0.1,0.25))
  ax.axis("equal")
  plt.colorbar(img)

In [13]:
#@title 浸水深ラスターの読み込みと確認
depth_map = np.load(file_flooddepth.format(int(maxdepth*100)))
depthmap = depth_map["floodmap_depth"]

if flg_checkDips:
  ax =plt.subplot(1,1,1)
  img = ax.contourf(lons,lats, depthmap)#,levels=np.arange(0,maxdepth+0.1,0.25))
  ax.axis("equal")
  plt.colorbar(img)

In [14]:
#@title 浸水カテゴリラスターの生成と確認
classificationmap = np.zeros_like(depthmap, dtype="int32")
classification_levels = [0,0.5,5,10,20]
#classificationmap[np.where(depthmap >= classification_levels[2])] = len(classification_levels)
#classificationmap[np.where(depthmap < classification_levels[1])] = len(classification_levels)
for ii, vv in enumerate(classification_levels[::-1][:-1]):
  print(ii, vv)
  classificationmap[np.where(depthmap < vv)] = len(classification_levels) - ii

if flg_checkDips:
  ax =plt.subplot(1,1,1)
  img = ax.contourf(lons,lats, classificationmap,levels=np.arange(-1,len(classification_levels)+1))
  ax.axis("equal")
  plt.colorbar(img)

0 20
1 10
2 5
3 0.5


## アップロード用タイルの生成

In [15]:
#@title XYZタイル分けされた点郡NPZファイルの生成関数
#print(len(lats)/256.0, len(lons)/256.0)

def save_tile_npz(lats,lons,map, classificationmap, zoom=15, dst_dir ="temp/"):
  dir_zoom = dst_dir + f"{zoom}/"
  if not os.path.exists(dir_zoom):
    os.makedirs(dir_zoom)
  res_lons = np.array([])
  res_lats = np.array([])
  res_maps = np.array([])
  res_class = np.array([])
  print(f" / total loop num: {len(lats)/256}")
  for jj, tmp_lat in progressbar(enumerate(lats[::256])):
    idj = jj*256
    sublats = lats[idj:idj+256]
    for ii, tmp_lon in enumerate(lons[::256]):
      idi = ii*256
      sublons = lons[idi:idi+256]
      xx,yy  = pfsl.calc_xyz_from_lonlat(tmp_lon, tmp_lat, zoom)
      #print(ii,jj, xx, yy)
      submap = map[idj:idj+256,idi:idi+256]
      subclass = classificationmap[idj:idj+256,idi:idi+256]
      grd_lons, grd_lats  = np.meshgrid(sublons,sublats)
      flt_lons = grd_lons.ravel()
      flt_lats = grd_lats.ravel()
      flt_map = submap.ravel()
      flt_class = subclass.ravel()
      #selected = [[lon,lat,dem] for lon, lat, dem in zip(flt_lons, flt_lats,flt_map) if not np.isnan(dem)]
      #print(flt_lons[0:5])
      #print(flt_lats[0:5])
      #print(flt_map[0:5])
      #print(ii, jj, f"{subzoom}-{xx}-{yy}", f"{subzoom}-{chk_tile[0]}-{chk_tile[1]}")
      if not os.path.exists(f"{dir_zoom}{xx}"):
        os.mkdir(f"{dir_zoom}{xx}")
      np.savez(f"{dir_zoom}{xx}/{yy}.npz", lons=flt_lons, lats=flt_lats, dem=flt_map, classification=flt_class)
      res_lons = np.append(res_lons, flt_lons)
      res_lats = np.append(res_lats, flt_lats)
      res_maps = np.append(res_maps, flt_map)
      res_class = np.append(res_class, flt_class)
  return res_lons, res_lats, res_maps, res_class


In [16]:
test_lons, test_lats, test_maps, test_class = save_tile_npz(lats,lons,geomap, classificationmap,dst_dir="ellipsoid/")

/ |#                                                  | 0 Elapsed Time: 0:00:00

 / total loop num: 16.0


| |        #                                         | 15 Elapsed Time: 0:01:30


In [17]:
#shutil.make_archive(path_upload+"flood_xyznpz","zip", root_dir = "ellipsoid")

In [18]:
#@title 点郡確認用
if flg_checkDips:
  plt_lons = test_lons[np.where(~np.isnan(test_maps))]
  plt_lats = test_lats[np.where(~np.isnan(test_maps))]
  plt.scatter(plt_lons,plt_lats, s=0.5)
  plt.axis("equal")

In [19]:
tilecnv = FloodTo3dtiles()

In [20]:
# 出力先のディレクトリが存在していたら消すこと

In [21]:
tilecnv.convert("ellipsoid/", path_flood3Dtile_tmp)

 100.0 % in 56 sec [est. time left: 0 sec]      

In [22]:
shutil.make_archive(file_flood3Dtile_zip[:-4], 'zip', path_flood3Dtile_tmp)

'/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2-rev2/ForUpload/flood_ellipsoid_3dtile.zip'

In [23]:
from plateauutils.flood_converter.flood_to_png import FloodToPng
p = FloodToPng("ellipsoid/")
p.parse(path_deltphtile_tmp)

In [24]:
shutil.make_archive(file_floodXYZtile_zip[:-4], 'zip', path_deltphtile_tmp)

'/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2-rev2/ForUpload/flood_depth_xyztile.zip'

## Re:EarthCMSへのアップロード

In [25]:
#@title Re:Earth CMSへのアップロード
if integrationKey is not None and projectId is not None:
  configuration = reearthcmsapi.Configuration(
      host = "https://api.cms.test.reearth.dev/api",
      access_token = integrationKey
  )

  client = reearthcmsapi.ApiClient(configuration)

  api_client = reearthcmsapi.ApiClient(configuration)
  api_instance = assets_project_api.AssetsProjectApi(api_client)

  # Create an instance of the API class
  api_instance = assets_project_api.AssetsProjectApi(api_client)
  path_params = {
      'projectId': projectId,
  }
  for ff in [file_flood3Dtile_zip, file_floodXYZtile_zip, file_building_csv]:
    body = dict(
        file=open(ff, 'rb'),
        skip_decompression=False,
    )
    try:
        # Create an new asset.
        api_response = api_instance.asset_create(
            path_params=path_params,
            body=body,
        )
    except reearthcmsapi.ApiException as e:
        print("Exception when calling AssetsProjectApi->asset_create: %s\n" % e)
  print("Re:Earth CMSへのアップロードが完了しました。")
else:
  print("Re:Earth CMSへのアップロードは行われませんでした。")

Re:Earth CMSへのアップロードは行われませんでした。
